# Variational Bayesian inference (MNIST)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.append('..')
sys.path.append('../../torchutils')

In [ ]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import PIL
import torch
import torch.nn as nn
import torch.distributions as dist
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms

from torchutils import Classification, confusion_matrix

from vartorch import (
    VariationalClassification,
    VariationalLinear,
    anomaly_score
)

In [ ]:
# set random seeds manually
np.random.seed(1223334444)
_ = torch.manual_seed(55555)

## Data import

In [ ]:
preprocessor = transforms.Compose([
    transforms.ToTensor()
])

In [ ]:
data_path = Path.home() / 'Data'

train_set = datasets.MNIST(
    data_path,
    train=True,
    transform=preprocessor,
    download=True
)

val_set = datasets.MNIST(
    data_path,
    train=False,
    transform=preprocessor,
    download=True
)

print('No. train images:', len(train_set))
print('No. val. images:', len(val_set))

In [ ]:
batch_size = 128

train_loader = DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True
)

val_loader = DataLoader(
    val_set,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False
)

print('No. train batches:', len(train_loader))
print('No. val. batches:', len(val_loader))

In [ ]:
images, labels = next(iter(train_loader))
print('Images shape:', images.shape)
print('Labels shape:', labels.shape)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(5, 3))
for idx, ax in enumerate(axes.ravel()):
    image = images[idx,0].numpy()
    ax.imshow(image.clip(0, 1), cmap='gray')
    ax.set_title(train_set.classes[labels[idx]])
    ax.set(xticks=[], yticks=[], xlabel='', ylabel='')
fig.tight_layout()

## Standard training

In [ ]:
# create logistic regression model
# model1 = nn.Sequential(
#     nn.Flatten(),
#     nn.Linear(in_features=28*28, out_features=10),
# )

In [ ]:
# create small CNN with linear classifier
model1 = nn.Sequential(
    nn.Conv2d(in_channels=1, out_channels=4, kernel_size=(5, 5), padding=2),
    nn.LeakyReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(in_channels=4, out_channels=8, kernel_size=(5, 5), padding=2),
    nn.LeakyReLU(),
    nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(in_features=7*7*8, out_features=10)
)

In [ ]:
criterion = nn.CrossEntropyLoss(reduction='mean')
optimizer = torch.optim.Adam(model1.parameters(), lr=0.01)

point_model = Classification(
    model1,
    criterion,
    optimizer,
    train_loader,
    val_loader
)

In [ ]:
point_history = point_model.training(
    num_epochs=100,
    log_interval=100
)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(np.array(point_history['train_loss']), label='train', alpha=0.7)
ax.plot(np.array(point_history['val_loss']), label='val.', alpha=0.7)
ax.set(xlabel='epoch', ylabel='loss')
ax.set_xlim((0, point_history['num_epochs']))
ax.legend()
ax.grid(visible=True, which='both', color='lightgray', linestyle='-')
ax.set_axisbelow(True)
fig.tight_layout()

In [ ]:
point_train_loss, point_train_acc = point_model.test(train_loader)
point_val_loss, point_val_acc = point_model.test(val_loader)

print('Train loss: {:.4f}'.format(point_train_loss))
print('Val. loss: {:.4f}'.format(point_val_loss))
print('\nTrain acc.: {:.4f}'.format(point_train_acc))
print('Val. acc.: {:.4f}'.format(point_val_acc))

In [ ]:
confmat = confusion_matrix(point_model, val_loader)
print('Confusion matrix:\n{}'.format(confmat))

## Variational inference

In [ ]:
# create variational logistic regression model
# model2 = nn.Sequential(
#     nn.Flatten(),
#     VariationalLinear(in_features=28*28, out_features=10),
# )

In [ ]:
# create small CNN with variational linear classifier
model2 = nn.Sequential(
    nn.Conv2d(in_channels=1, out_channels=4, kernel_size=(5, 5), padding=2),
    nn.LeakyReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(in_channels=4, out_channels=8, kernel_size=(5, 5), padding=2),
    nn.LeakyReLU(),
    nn.MaxPool2d(2),
    nn.Flatten(),
    VariationalLinear(in_features=7*7*8, out_features=10)
)

In [ ]:
optimizer = torch.optim.Adam(model2.parameters(), lr=0.01)

post_model = VariationalClassification(
    model2,
    likelihood_type='Categorical'
)

post_model.compile_for_training(
    optimizer,
    train_loader,
    val_loader
)

In [ ]:
post_history = post_model.training(
    num_epochs=100,
    num_samples=10,
    log_interval=100
)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(-np.array(post_history['train_loss']), label='train', alpha=0.7)
ax.plot(-np.array(post_history['val_loss']), label='val.', alpha=0.7)
ax.set(xlabel='epoch', ylabel='ELBO')
ax.set_xlim((0, post_history['num_epochs']))
ax.legend()
ax.grid(visible=True, which='both', color='lightgray', linestyle='-')
ax.set_axisbelow(True)
fig.tight_layout()

In [ ]:
post_train_loss = post_model.test_loss(train_loader)
post_train_acc = post_model.test_acc(train_loader)

post_val_loss = post_model.test_loss(val_loader)
post_val_acc = post_model.test_acc(val_loader)

print('Train loss: {:.4f}'.format(post_train_loss))
print('Val. loss: {:.4f}'.format(post_val_loss))
print('\nTrain acc.: {:.4f}'.format(post_train_acc))
print('Val. acc.: {:.4f}'.format(post_val_acc))

In [ ]:
confmat = confusion_matrix(post_model, val_loader, num_samples=100)
print('Confusion matrix:\n{}'.format(confmat))

## Example predictions

In [ ]:
norm_set = val_set
norm_loader = val_loader

anom_set = datasets.KMNIST(data_path, train=False, transform=preprocessor, download=True)
# anom_set = datasets.FashionMNIST(data_path, train=False, transform=preprocessor, download=True)
# anom_set = TensorDataset(torch.rand(batch_size, 1, 28, 28), torch.zeros((batch_size,), dtype=torch.int64))

anom_loader = DataLoader(anom_set, batch_size=batch_size, shuffle=True)

norm_images, norm_labels = next(iter(norm_loader))
anom_images, anom_labels = next(iter(anom_loader))

In [ ]:
point_model.train(False)

with torch.no_grad():
    point_norm_probs = point_model.predict_proba(norm_images.to(point_model.device)).cpu()
    point_norm_entropy = dist.Categorical(probs=point_norm_probs).entropy()

    point_anom_probs = point_model.predict_proba(anom_images.to(point_model.device)).cpu()
    point_anom_entropy = dist.Categorical(probs=point_anom_probs).entropy()

In [ ]:
num_samples = 500

post_model.sample(True)
post_model.train(False)

with torch.no_grad():
    sampled_norm_logits = post_model.predict(norm_images.to(post_model.device), num_samples).cpu()
    sampled_norm_probs = torch.softmax(sampled_norm_logits, dim=1)

    post_norm_probs = torch.mean(sampled_norm_probs, axis=-1)
    post_norm_entropy = dist.Categorical(probs=post_norm_probs).entropy()

    sampled_anom_logits = post_model.predict(anom_images.to(post_model.device), num_samples).cpu()
    sampled_anom_probs = torch.softmax(sampled_anom_logits, dim=1)

    post_anom_probs = torch.mean(sampled_anom_probs, axis=-1)
    post_anom_entropy = dist.Categorical(probs=post_anom_probs).entropy()

In [ ]:
# plot point predictions (in distribution)
plot_ids = np.random.permutation(np.arange(len(images))) # random
# plot_ids = torch.argsort(point_norm_entropy, descending=False).data.numpy() # lowest entropy

fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(5, 6))
for idx, (ax1, ax2) in zip(plot_ids[:axes.shape[0]], axes):
    # image
    image = norm_images[idx,0].numpy()
    ax1.imshow(image.clip(0, 1), cmap='gray')
    ax1.set_title(
        '{}'.format(norm_set.classes[norm_labels[idx]])
        if hasattr(norm_set, 'classes') else 'random'
    )
    ax1.set(xticks=[], yticks=[], xlabel='', ylabel='')

    # probabilities
    ax2.bar(np.arange(10), point_norm_probs.data.numpy()[idx])
    ax2.set_title('$\pi(c|x,\hat{w})$')
    ax2.set(xticks=np.arange(10), ylim=(0, 1), xlabel='c')
    # ax2.text(0, 0.75, 'entropy: {:.2f}'.format(point_norm_entropy[idx]), alpha=0.5)

fig.suptitle('Point predictions (in distribution)')
fig.tight_layout()

In [ ]:
# plot point predictions (out of distribution)
plot_ids = np.random.permutation(np.arange(len(images))) # random
# plot_ids = torch.argsort(point_anom_entropy, descending=False).data.numpy() # lowest entropy

fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(5, 6))
for idx, (ax1, ax2) in zip(plot_ids[:axes.shape[0]], axes):
    # image
    image = anom_images[idx,0].numpy()
    ax1.imshow(image.clip(0, 1), cmap='gray')
    ax1.set_title(
        '{}'.format(anom_set.classes[anom_labels[idx]])
        if hasattr(anom_set, 'classes') else 'random'
    )
    ax1.set(xticks=[], yticks=[], xlabel='', ylabel='')

    # probabilities
    ax2.bar(np.arange(10), point_anom_probs.data.numpy()[idx])
    ax2.set_title('$\pi(c|x,\hat{w})$')
    ax2.set(xticks=np.arange(10), ylim=(0, 1), xlabel='c')
    # ax2.text(0, 0.75, 'entropy: {:.2f}'.format(point_anom_entropy[idx]), alpha=0.5)

fig.suptitle('Point predictions (out of distribution)')
fig.tight_layout()

In [ ]:
# plot posterior predictions (in distribution)
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(8, 6))

for idx, (ax1, ax2, ax3) in zip(plot_ids[:axes.shape[0]], axes):
    # image
    image = norm_images[idx,0].numpy()
    ax1.imshow(image.clip(0, 1), cmap='gray')
    ax1.set_title(
        '{}'.format(norm_set.classes[norm_labels[idx]])
        if hasattr(norm_set, 'classes') else 'random noise'
    )
    ax1.set(xticks=[], yticks=[], xlabel='', ylabel='')

    # violin plot
    # ax2.violinplot(sampled_norm_probs[idx,:,:], positions=np.arange(10))
    # ax2.set_title('$\pi(c|x,w)$, $w$ from $\pi(w|\mathcal{D})$')
    # ax2.set(xticks=np.arange(10), ylim=[0,1], xlabel='c')

    # histogram
    highest_ids = post_norm_probs[idx].data.numpy().argsort()[::-1][:3]
    for highest_idx in highest_ids:
        ax2.hist(
            sampled_norm_probs[idx,highest_idx,:].data.numpy(), bins=50,
            range=(0, 1), density=True, histtype='stepfilled', alpha=0.5
        )
    ax2.set_title('$\pi(c|x,w)$, $w$ from $\pi(w|\mathcal{D})$')
    ax2.set_xlim((0, 1))
    ax2.legend(['c={}'.format(c) for c in highest_ids], loc='upper center')
    ax2.grid(visible=True, which='both', color='lightgray', linestyle='-')
    ax2.set_axisbelow(True)

    # posterior predictive
    ax3.bar(np.arange(10), post_norm_probs[idx].data.numpy())
    ax3.set_title('$\pi(c|x,\mathcal{D}) = \int \pi(c|x,w) \pi(w|\mathcal{D}) dw$')
    ax3.set(xticks=np.arange(10), ylim=(0, 1), xlabel='c')
    # ax3.text(0, 0.75, 'entropy: {:.2f}'.format(post_norm_entropy[idx]), alpha=0.5)

fig.suptitle('Posterior predictions (in distribution)')
fig.tight_layout()

In [ ]:
# plot posterior predictions (out of distribution)
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(8, 6))

for idx, (ax1, ax2, ax3) in zip(plot_ids[:axes.shape[0]], axes):
    # image
    image = anom_images[idx,0].numpy()
    ax1.imshow(image.clip(0, 1), cmap='gray')
    ax1.set_title(
        '{}'.format(anom_set.classes[anom_labels[idx]])
        if hasattr(anom_set, 'classes') else 'random noise'
    )
    ax1.set(xticks=[], yticks=[], xlabel='', ylabel='')

    # violin plot
    # ax2.violinplot(sampled_anom_probs[idx,:,:], positions=np.arange(10))
    # ax2.set_title('$\pi(c|x,w)$, $w$ from $\pi(w|\mathcal{D})$')
    # ax2.set(xticks=np.arange(10), ylim=[0,1], xlabel='c')

    # histogram
    highest_ids = post_anom_probs[idx].data.numpy().argsort()[::-1][:3]
    for highest_idx in highest_ids:
        ax2.hist(
            sampled_anom_probs[idx,highest_idx,:].data.numpy(), bins=50,
            range=[0,1], density=True, histtype='stepfilled', alpha=0.5
        )
    ax2.set_title('$\pi(c|x,w)$, $w$ from $\pi(w|\mathcal{D})$')
    ax2.set_xlim((0, 1))
    ax2.legend(['c={}'.format(c) for c in highest_ids], loc='upper center')
    ax2.grid(visible=True, which='both', color='lightgray', linestyle='-')
    ax2.set_axisbelow(True)

    # posterior predictive
    ax3.bar(np.arange(10), post_anom_probs[idx].data.numpy())
    ax3.set_title('$\pi(c|x,\mathcal{D}) = \int \pi(c|x,w) \pi(w|\mathcal{D}) dw$')
    ax3.set(xticks=np.arange(10), ylim=(0, 1), xlabel='c')
    # ax3.text(0, 0.75, 'entropy: {:.2f}'.format(post_anom_entropy[idx]), alpha=0.5)

fig.suptitle('Posterior predictions (out of distribution)')
fig.tight_layout()

## Out-of-distribution detection

In [ ]:
norm_loader = val_loader

anom_set = datasets.KMNIST(data_path, train=False, transform=preprocessor, download=True)
# anom_set = datasets.FashionMNIST(data_path, train=False, transform=preprocessor, download=True)
# anom_set = TensorDataset(torch.rand(batch_size, 1, 28, 28), torch.zeros((batch_size,), dtype=torch.int64))

anom_loader = DataLoader(anom_set, batch_size=batch_size, shuffle=True)

In [ ]:
point_norm_entropy = anomaly_score(point_model, norm_loader, mode='entropy')
point_anom_entropy = anomaly_score(point_model, anom_loader, mode='entropy')

post_norm_entropy = anomaly_score(post_model, norm_loader, mode='entropy', num_samples=100)
post_anom_entropy = anomaly_score(post_model, anom_loader, mode='entropy', num_samples=100)

In [ ]:
# plot point entropy histogram
fig, ax = plt.subplots(figsize=(6, 4))
ax.hist(
    point_norm_entropy, bins=100, range=(0, 2), density=True,
    histtype='stepfilled', alpha=0.7, label='in distribution'
)
ax.hist(
    point_anom_entropy, bins=100, range=(0, 2), density=True,
    histtype='stepfilled', alpha=0.7, label='out of distribution'
)
ax.set(xlim=(0, 2), xlabel='entropy', ylabel='density')
ax.set_title('Point predictions')
ax.legend(loc='upper right')
ax.grid(visible=True, which='both', color='lightgray', linestyle='-')
ax.set_axisbelow(True)
fig.tight_layout()

In [ ]:
# plot posterior entropy histogram
fig, ax = plt.subplots(figsize=(6, 4))
ax.hist(
    post_norm_entropy, bins=100, range=(0, 2), density=True,
    histtype='stepfilled', alpha=0.7, label='in distribution'
)
ax.hist(
    post_anom_entropy, bins=100, range=(0, 2), density=True,
    histtype='stepfilled', alpha=0.7, label='out of distribution'
)
ax.set(xlim=(0, 2), xlabel='entropy', ylabel='density')
ax.set_title('Posterior predictive')
ax.legend(loc='upper right')
ax.grid(visible=True, which='both', color='lightgray', linestyle='-')
ax.set_axisbelow(True)
fig.tight_layout()